# Analyse fréquentielle intégratrice

## Activité de transfert - 21 juillet 2022

préparée par :
- Duy Anh Alexandre
- Gabriel Gobeil
- Jonathan Jalbert

## Application 3 : Analyse fréquentielle des simulations hydroclimatiques

Dans cette application, vous reprendrez les grandes étapes du calepin 3-SimulationFrequencyAnalysis.ipynb pour le tronçon de la rivière Famine : SLSO00160

Vous n'avez qu'à compléter les cellules où il est spécificié **#TODO**.

---
# 1. Chargement des librairies et des fonctions utiles <a name="preparation"></a>

In [ ]:
# Chargement des librairies
using CSV, DataFrames, Dates, NetCDF
using Distributions, Extremes, LinearAlgebra, Mamba, Random, Statistics
using Gadfly
using ProgressMeter, StatsBase

using Test

using ErrorsInVariablesExtremes

In [ ]:
include("3-utils.jl")
include("3-hierarchicalbayesmodel.jl")
include("3-hierarchicalbayeseva.jl");

---
# 2. Chargement des données <a name="data"></a>

## 2.1 Chargement des simulations hydroclimatiques <a name="data_loading"></a>

#### Description des données

Les données se trouvent dans le fichier ```A2020_Simulation_Hydroclimatique_QMA_SLSO00160.nc``` qui contient 1080 simulations de débits produites par le modèle HYDROTEL en utilisant 6 configurations pour 180 simulations climatiques provenant de différents couples GCM-RCM de 1956 à 2100 (145 ans).

#### Chargement des données

In [ ]:
filename = "../0-Data/0-Simulations/A2020_Simulation_Hydroclimatique_QMA_SLSO00160.nc"
data = load_discharge_simulations(filename)
first(data,1)

## 2.2. Sélection des simulations hydroclimatiques <a name="data_selection"></a>

In [ ]:
#TODO: Récupérer les lignes de votre ensemble préféré avec la commande `filter()`. 
# Pour ClimEX, décommentez la ligne suivante

subdata = filter(row -> row[:Ensemble] == "CMX", data)


---
# 3. Modélisation bayésienne des maxima annuels simulés <a name="modelisation"></a>


## 3.1 Modèle stationnaire  <a name="mstat"></a>

### 3.1.1 Définition du modèle

#### Couche des données

\begin{equation*}
    f_{(X_{ij}|\mu_i, \phi_i, \xi_i)}(x_{ij}) = \mathcal{G}EV(x_{ij} \mid \mu_i, \sigma_i, \xi_i)
\end{equation*}

où $x_{ij}$ correspond au maximum annuel de l'année j du membre i.

#### Couche latente

La loi normale est utilisée pour modéliser la dépendance des paramètres de la loi GEV entre les différents membres. Par exemple, pour le paramètre de localisation, on suppose que les paramètres de localisation de chacun des membres  proviennent tous d'une même distribution :

\begin{equation*}
    f_{(\mu_i| \nu_0, \tau_0)} = \mathcal{N}(\mu_{i} \mid \nu_0, \tau_0)
\end{equation*}

#### Loi *a priori*

Une loi *a priori* vague est utilisée pour les paramèetres de la couche latente. Par exemple pour $(\nu_0, \tau_0)$, la loi *a priori* est la suivante :

\begin{equation*}
    f_{(\nu_0, \tau_0)}(\nu_0, \tau_0) \propto \mathcal{I}nverse\mathcal{G}amma(0.01, 0.01)
\end{equation*}


#### Structure `HierarchicalBayesModel`

On encapsule les données et la loi *a priori* des hyperparamètres dans la structure `HierarchicalBayesModel` :

In [ ]:
# Préparation de la liste de modèles BlockMaxima avec les données
M₁ = BlockMaxima[]
for i in 1:size(subdata,1)
    push!(M₁, BlockMaxima(Variable("y", subdata[i, :Discharges])))
end

# Préparation du modèle hiérachique 
model1 = HierarchicalBayesModel(M₁, [Flat(), InverseGamma(0.01, 0.01)])

### 3.1.2 Génération d'un échantillon de la loi *a posteriori* par MCMC

La fonction `fitbayes` permet de générer un échantillon de la loi *a posteriori* des paramètres du modèle bayésien défini précédement. 

La fonction prend en entrée une structure `HierarchicalBayesModel`.

In [ ]:
#TODO: Générez un échantillon aléatoire de la loi a posteriori du modèle hiérarchique bayésien


La fonction `transform` peut être utilisée pour obtenir le modèle ajusté avec les paramètres à l'échelle originale :

In [ ]:
#TODO: Transformer à leurs échelles originales les variables du modèle avec la fonction `transform()`.


### 3.1.3 Affichage de la trace des hyperparamètres

Permet de constater si l'algorithme MCMC a bien convergé. Permet aussi de voir l'échelle des différents paramètres de la loi GEV.

In [ ]:
#TODO: Affichez la trace de certains paramètres


### 3.1.4 Adéquation de la loi GEV aux membres de l'ensemble

#### Adéquation pour un membre

In [ ]:
#TODO: Affichez les figures diagnostiques pour un membre de l'ensemble


### 3.1.5 Estimation des niveaux de retour

On estime les niveaux de retour avec l'ensemble des membres. On obtient alors un échantillon aléatoire de la loi prédictive du niveau de retour.

In [ ]:
# Période de retour (en année)
T = 20

#TODO: Calculez le quantile correspondant à cette période de retour avec la fonction `returnlevel()`.


In [ ]:
# Affichez l'histogramme des quantiles précédents. Il s'agit de la loi prédictive du quantile.


## 3.2 Modèle non-stationnaire  <a name="mnstat"></a>

Le paramètre de localisation de la loi GEV est une fonction de la concentration de GES dans l'atmosphère.

\begin{equation*}
    f_{(X_{ij}|\mu_i, \phi_i, \xi_i)}(x_{ij}) = \mathcal{G}EV(x_{ij} \mid \mu_{ij}, \sigma_i, \xi_i)
\end{equation*}

avec

\begin{equation*}
    \mu_{ij} = \mu_{0i} + \mu_{1i} \times u_j
\end{equation*}

où

$u_{j}$ correspond à la concentration équivalente de CO₂ dans l'atmosphère.


### 3.2.1 Chargement des variables exlicatives

In [ ]:
 # Chargement des données de CO₂
co2data = CSV.read("../0-Data/0-RCP/RCPdata.csv", DataFrame);

# Ajout de la covariable à la structure de données 
function get_co2(co2data, scenario, years)
    
    data = select(co2data, :Year, Symbol(scenario))
    filter!(row -> row.Year in years, data)
    
    return data[!, Symbol(scenario)]
end

subdata[!, Symbol("CO₂")] = get_co2.(Ref(co2data), subdata.Scenario, subdata.Years);

### 3.2.2 Définition du modèle

In [ ]:
# Préparation de la liste de modèles BlockMaxima avec les données
M₂ = BlockMaxima[]
for i in 1:size(subdata,1)
    push!(M₂, BlockMaxima(Variable("y", subdata[i, :Discharges]),
            locationcov = [Variable(subdata[i,:Scenario], subdata[i, :CO₂])]))
end

# Préparation du modèle hiérachique 
model2 = HierarchicalBayesModel(M₂, [Flat(), InverseGamma(0.01, 0.01)])

### 3.2.3 Génération d'un échantillon de la loi *a posteriori* par MCMC

In [ ]:
#TODO: Générez un échantillon de la loi a posteriori des paramètres.


In [ ]:
#TODO: Transformer à leurs échelles originales les variables du modèle avec la fonction `transform()`.


### 3.2.3 Affichage de la trace des hyperparamètres

In [ ]:
#TODO: Affichez la trace de certains paramètres


### 3.2.4 Adéquation de la loi GEV aux membres de l'ensemble

On ne peut pas tracer de return level plot ici car le modèle est non stationnaire.

#### Adéquation pour un membre

In [ ]:
#TODO: Affichez les figures diagnostiques pour un membre de l'ensemble


### 3.2.5 Estimation des niveaux de retour effectifs

On estime les niveaux de retour effectifs avec l'ensemble des membres. On obtient alors un échantillon aléatoire de la loi prédictive des niveaux de retour pour chacune des années. 

In [ ]:
# Période de retour
T = 20

#TODO: Calculez les niveaux de retour effectifs correspondants. 


In [ ]:
#TODO: Calculez les intervalles de crédibilité à 95% des niveaux de retour effectifs. 


---
# 4. Sélection de modèle  <a name="selection"></a>

La fonction `dic`permet de calculer le critère d'information de déviation (DIC) :

In [ ]:
#TODO: Calculez le dic du modèle stationnaire et du modèle non stationnaire avec la fonction `dic()`.


In [ ]:
#TODO: Choisissez le meilleur modèle, celui qui minimise le DIC.


---
# 5. Enregistrement du modèle  <a name="enregistrement"></a>

Pour pouvoir utiliser les résultats de l'ajustement du modèle dans les parties suivantes de l'activité sans avoir à tout recalculer, il est utile d'enregistrer les résultats. La librairie Julia de base `Serialization` permet de sérialiser la structure `HierarchicalBayesEVA` et de l'exporter sous forme de fichier .txt qui pourra être désérialiser dans les parties suivantes.

In [ ]:
using Serialization

io = open("2-Results/Famine_sim.txt", "w");

#TODO: Remplacez fm1 par le nom de votre meilleur modèle dans la commande suivante et exécutez-la.
serialize(io, fm1);

close(io);